In [58]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time
import random

my_header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

# Single Product Pages

In [8]:
# List of product links scraped from locally downloaded file
file_path = "C:/Users/moth1/Desktop/3M_products.html"
text = BeautifulSoup(open(file_path), "html.parser")
# text = BeautifulSoup(response.text, 'html.parser')

In [357]:
%%time

def sleep():
    time.sleep(random.randint(10,15))

df_total = pd.DataFrame()
mfg = '3M'
group_title = 'Personal Protective Equipment'

#           (group)             >    (product line)   >     (product)
# Personal Protective Equipment >    safety glasses   >  SF407SGAF-BLU

multiple_product_list = []
i = 0

file_path = "C:/Users/moth1/Desktop/3M_products.html"
text = BeautifulSoup(open(file_path), "html.parser")
search_list = text.find_all('li', {'class':'MMM--tileLayout-col'})
# search_list = search_list[i:]
for search in search_list:
    search_header = search.find('a', {'class':'MMM--tileTitle'})
    print('Scraping #', i, ': ', search_header.get_text())
    if search.get_text().find('Options') > 0:
        print('\tSaved for later.')
        multiple_product_list.append((search_header.text, search_header['href']))      
    else:
#         product_title = search_header.text
        product_url = search_header['href']
        product_response = requests.get(product_url, headers=my_header)
        print('\tResponse code: ', product_response.status_code)
        product_text = BeautifulSoup(product_response.text, 'html.parser')
        
        product_title = product_text.find('h1', {'class':'MMM--hdg'})
        if product_title:
            product_title = product_title.text.replace('â„¢', '').replace('™', '').replace('\n', ' ')
        product_header = product_text.find('div',{'class':'MMM--contentDetailContain-bd'}).get_text().strip()
        product_header = product_header.replace('\xa0', ': ').replace(',:', ' , ')
        if product_header.find('Number') > -1:
            product_number = re.search("(?<=Number: )(\S+)", product_header).group()
        elif  product_header.find('3M ID') > -1:
            product_number = re.search("(?<=3M ID: )(\S+)", product_header).group()
        if product_header.find('UPC') > -1:
            product_upc = re.search("(?<=UPC: )(\d+)", product_header).group()
            
        spec_table = product_text.find('div',{'class':'productSpecs'}).find_all('tr')
        spec_table = spec_table[1:]
        description = '; '.join([t.get_text().strip().replace('\n\n\n\n', ':') for t in spec_table])
        if description.find('Product Type') > -1:
            product_line = re.search("(?<=Product Type:)(.*)", description).group().split(';')[0]
        elif description.find('Segment') > -1:
            product_line = re.search("(?<=Segment:)(.*)", description).group().split(';')[0]
        description = description.replace('â„¢', '').replace('™', '').replace('\n', '')
        df_product = pd.DataFrame({'mfg':[mfg],
                                   'group':[group_title],
                                   'product_line':[product_line],
                                   'product_title':[product_title],
                                   'product_number':[product_number],
                                   'upc':[product_upc],
                                   'description':[description],
                                   'url':[product_url]})
        print('\tProduct Title: ', product_title[0:50])
        print('\tDescription: ', description[0:50])
        print('\tURL: ', product_url)
        df_total = df_total.append(df_product)
        
    i += 1
    sleep()
    
print('Done!')
df_total = df_total.reset_index(drop=True)
df_total.head()

Scraping # 0 :  3Mâ„¢ Integrated Protective Eyewear
	Saved for later.
Scraping # 1 :  3Mâ„¢ SecureFitâ„¢ Safety Glasses SF401SGAF-RED, ...
	Response code:  200
	Product Title:  3M SecureFit Safety Glasses SF401SGAF-RED, Red/Gra
	Description:  Brand:SecureFit; Case Quantity:20; Comfort Feature
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-SecureFit-Safety-Glasses-SF401SGAF-RED-Red-Gray-Clear-Scotchgard-Anti-fog-Lens-20-EA-Case/?N=5002385+8707795+8720539+3290617453&rt=rud
Scraping # 2 :  3Mâ„¢ Particulate Respirator 8511, N95 80 EA/Case
	Response code:  200
	Product Title:  3M Particulate Respirator 8511, N95 80 EA/Case
	Description:  Acid Gas Reduction:No; Adjustable Buckle Straps:No
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Particulate-Respirator-8511-N95-80-EA-Case/?N=5002385+8707795+8720539+3294780243&rt=rud
Scraping # 3 :  3Mâ„¢ SecureFitâ„¢ Safety Glasses SF401SGAF-BLU, ...
	Response code:  200
	Product Title:  3M SecureFit Safety Glass

Scraping # 22 :  3Mâ„¢ Versafloâ„¢ High Capacity Battery ...
	Response code:  200
	Product Title:  3M Versaflo High Capacity Battery TR-632/37348(AAD
	Description:  Battery Life:8 - 18 Hours; Battery Type:Lithium Io
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Versaflo-High-Capacity-Battery-TR-632-37348-AAD-for-TR-600-PAPR-1-EA-Case/?N=5002385+8707795+3292439749&rt=rud
Scraping # 23 :  3Mâ„¢ Particulate Respirator 8200/07023(AAD), N95 ...
	Response code:  200
	Product Title:  3M Particulate Respirator 8200/07023(AAD), N95 160
	Description:  Acid Gas Reduction:No; Adjustable Buckle Straps:No
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Particulate-Respirator-8200-07023-AAD-N95-160-EA-Case/?N=5002385+8707795+8720539+3294780149&rt=rud
Scraping # 24 :  3Mâ„¢ PELTORâ„¢ Airplane PTT Adapter, FL5006 ...
	Response code:  200
	Product Title:  3M PELTOR Airplane PTT Adapter, FL5006 GB, 1 EA/CS
	Description:  Auxiliary Port:J11; Brand:PELTOR; Case Quant

Scraping # 44 :  3Mâ„¢ W90 Series Faceshields & Visors
	Saved for later.
Scraping # 45 :  3Mâ„¢ Versafloâ„¢ Healthcare PAPR Kit TR-600-HKS, 1 ...
	Response code:  200
	Product Title:  3M Versaflo Healthcare PAPR Kit TR-600-HKS, 1 ea/C
	Description:  Battery Life:4 - 12 Hours; Battery Type:Standard C
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Versaflo-Healthcare-PAPR-Kit-TR-600-HKS-1-ea-Case/?N=5002385+8707795+3292194436&rt=rud
Scraping # 46 :  3Mâ„¢ Versafloâ„¢ Healthcare PAPR Kit TR-600-HKL, 1 ...
	Response code:  200
	Product Title:  3M Versaflo Healthcare PAPR Kit TR-600-HKL, 1 ea/C
	Description:  Battery Life:4 - 12 Hours; Battery Type:Standard C
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Versaflo-Healthcare-PAPR-Kit-TR-600-HKL-1-ea-Case/?N=5002385+8707795+3292194430&rt=rud
Scraping # 47 :  3Mâ„¢ Breathe Easyâ„¢ Powered Air Purifying Respirator Lithium Battery Pack
520-04-57R01, 1 ...
	Response code:  200
	Product Title:  3M Breathe E

Scraping # 65 :  3Mâ„¢ SecureFitâ„¢ 400 Series Safety Glasses SF401XAS-GRN, Green/Black,
Clear Anti-Scratch ...
	Response code:  200
	Product Title:  3M SecureFit 400 Series Safety Glasses SF401XAS-GR
	Description:  Anti Fog Lens:Yes; Brand:SecureFit; Case Quantity:
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-SecureFit-400-Series-Safety-Glasses-SF401XAS-GRN-Green-Black-Clear-Anti-Scratch-Lens-20-EA-Case/?N=5002385+8707795+8720539+3290356038&rt=rud
Scraping # 66 :  3Mâ„¢ SecureFitâ„¢ 400 Series Safety Glasses SF402XAS-GRN, Green/Black, Gray
Anti-Scratch ...
	Response code:  200
	Product Title:  3M SecureFit 400 Series Safety Glasses SF402XAS-GR
	Description:  Anti Fog Lens:No; Brand:SecureFit; Case Quantity:2
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-SecureFit-400-Series-Safety-Glasses-SF402XAS-GRN-Green-Black-Gray-Anti-Scratch-Lens-20-EA-Case/?N=5002385+8707795+8720539+3290356007&rt=rud
Scraping # 67 :  3Mâ„¢ SecureFitâ„¢ 400 Series Safet

Scraping # 84 :  3Mâ„¢ Battery Pack 520-01-02R01, NiCd, Intrinsically ...
	Response code:  200
	Product Title:  3M Battery Pack 520-01-02R01, NiCd, Intrinsically 
	Description:  Battery Life:8 Hours; Battery Type:NiCd - Recharge
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Battery-Pack-520-01-02R01-NiCd-Intrinsically-Safe-1-EA-Case/?N=5002385+8707795+3294776207&rt=rud
Scraping # 85 :  3Mâ„¢ Disposable Protective Coverall 4520
	Saved for later.
Scraping # 86 :  3Mâ„¢ Versafloâ„¢ 4-Station Battery Charger Kit ...
	Response code:  200
	Product Title:  3M Versaflo 4-Station Battery Charger Kit TR-644N/
	Description:  Brand:Versaflo; Case Quantity:1/case; Color Family
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Versaflo-4-Station-Battery-Charger-Kit-TR-644N-37351-AAD-1-EA-Case/?N=5002385+8707795+3292439748&rt=rud
Scraping # 87 :  3Mâ„¢ 4-Station Battery Charger Kit TR-344N, for Versafloâ„¢ TR-300 &
Speedglasâ„¢ TR-300-SG PAPR, 1 ...
	Response cod

Scraping # 113 :  3Mâ„¢ E-A-Rsoftâ„¢ Yellow Neonsâ„¢ Probed Test ...
	Response code:  200
	Product Title:  3M E-A-Rsoft Yellow Neons Probed Test Plugs 393-20
	Description:  Brand:3M; Case Quantity:50; Color Family:Yellow; C
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-E-A-Rsoft-Yellow-Neons-Probed-Test-Plugs-393-2000-50-50-EA-Case/?N=5002385+8707795+8720539+3294754027&rt=rud
Scraping # 114 :  3Mâ„¢ Versafloâ„¢ Battery Charger Cradle TR-640, ...
	Response code:  200
	Product Title:  3M Versaflo Battery Charger Cradle TR-640, for TR-
	Description:  Brand:Versaflo; Case Quantity:1/pack; Compatible R
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Versaflo-Battery-Charger-Cradle-TR-640-for-TR-600-800-Series-PAPR-1-case/?N=5002385+8707795+8720539+3292438374&rt=rud
Scraping # 115 :  3Mâ„¢ Virtuaâ„¢ Protective Eyewear 11331-00000-20 Blue Mirror HC Lens, Blue
Mirror Temple 20 ...
	Response code:  200
	Product Title:  3M Virtua Protective Eyewear 11331-0

Scraping # 133 :  3Mâ„¢ Disposable Protective Coverall 4510CS Series
	Saved for later.
Scraping # 134 :  3Mâ„¢ Carrying Bag 520-01-81 1 EA/Case
	Response code:  200
	Product Title:  3M Carrying Bag 520-01-81 1 EA/Case
	Description:  Brand:3M; Compatible Respirator:Airstream; Headgea
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Carrying-Bag-520-01-81-1-EA-Case/?N=5002385+8707795+3294750346&rt=rud
Scraping # 135 :  3Mâ„¢ E-A-Râ„¢ Classicâ„¢ Probed Test Plugs ...
	Response code:  200
	Product Title:  3M E-A-R Classic Probed Test Plugs 393-2003-50, 50
	Description:  Brand:E-A-R; Case Quantity:50; Color Family:Yellow
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-E-A-R-Classic-Probed-Test-Plugs-393-2003-50-50-EA-Case/?N=5002385+8707795+8720539+3294754026&rt=rud
Scraping # 136 :  3Mâ„¢ PELTORâ„¢ Right Side Boom Mounting ...
	Response code:  200
	Product Title:  3M PELTOR Right Side Boom Mounting Y-Harness FL6AB
	Description:  Brand:Peltor; Intrinsica

Scraping # 155 :  3Mâ„¢ E-A-Râ„¢ UltraFitâ„¢ Silicone Earplugs 340-3002, Corded, ...
	Response code:  200
	Product Title:  3M E-A-R UltraFit Silicone Earplugs 340-3002, Cord
	Description:  Attenuation:24 Decibel; Brand:E-A-R; Case Quantity
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-E-A-R-UltraFit-Silicone-Earplugs-340-3002-Corded-with-Carry-Case-80-Pair-Box-4-Boxes-EA-Case-320-Pair-Case/?N=5002385+8707795+3291175082&rt=rud
Scraping # 156 :  3Mâ„¢ PELTORâ„¢ MT Series In-Line PTT Adaptor FL5035, Kenwood, ...
	Response code:  200
	Product Title:  3M PELTOR MT Series In-Line PTT Adaptor FL5035, Ke
	Description:  Brand:Peltor; Color Family:Black; Compatible Radio
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-PELTOR-MT-Series-In-Line-PTT-Adaptor-FL5035-Kenwood-1-ea-cs/?N=5002385+8707795+3294754553&rt=rud
Scraping # 157 :  3Mâ„¢ Disposable Protective Coverall 4535
	Saved for later.
Scraping # 158 :  3Mâ„¢ Breathe Easyâ„¢ Turbo Powered Air Purifying

Scraping # 179 :  15-0099-08N6 ADFLO MULTIPLE BATTERY SMART CHARGER ...
	Response code:  200
	Product Title:  15-0099-08N6 ADFLO MULTIPLE BATTERY SMART CHARGER 
	Description:  Case Quantity:1; Features:Respiratory Accessory; P
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/15-0099-08N6-ADFLO-MULTIPLE-BATTERY-SMART-CHARGER-1-CASE/?N=5002385+8707795+3294754780&rt=rud
Scraping # 180 :  3Mâ„¢ Versafloâ„¢ PAPR Unit TR-302N, 1 EA/Case
	Response code:  200
	Product Title:  3M Versaflo PAPR Unit TR-302N, 1 EA/Case
	Description:  Brand:Versaflo; Cartridge or Filter Included:No; C
	URL:  https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Versaflo-PAPR-Unit-TR-302N-1-EA-Case/?N=5002385+8707795+3294754016&rt=rud
Scraping # 181 :  3Mâ„¢ E-A-Râ„¢ Classicâ„¢ Earplugs VP312-1201, Uncorded, Vending ...
	Response code:  200
	Product Title:  3M E-A-R Classic Earplugs VP312-1201, Uncorded, Ve
	Description:  Attenuation:29 Decibel; Brand:E-A-R; Case Quantity
	URL:  https://www.3m.

,mfg,group,product_line,product_title,product_number,upc,description,url
0,3M,Personal Protective Equipment,Safety Glasses,"3M SecureFit Safety Glasses SF401SGAF-RED, Red...",SF401SGAF-RED,50051131277156,Brand:SecureFit; Case Quantity:20; Comfort Fea...,https://www.3m.com/3M/en_US/company-us/all-3m-...
1,3M,Personal Protective Equipment,Dust and other Particles,"3M Particulate Respirator 8511, N95 80 EA/Case",8511,50051138543438,Acid Gas Reduction:No; Adjustable Buckle Strap...,https://www.3m.com/3M/en_US/company-us/all-3m-...
2,3M,Personal Protective Equipment,Protective Eyewear,"3M SecureFit Safety Glasses SF401SGAF-BLU, Blu...",SF401SGAF-BLU,50051131277286,Anti Fog Lens:Yes; Brand:SecureFit; Case Quant...,https://www.3m.com/3M/en_US/company-us/all-3m-...
3,3M,Personal Protective Equipment,Protective Eyewear,"3M SecureFit Safety Glasses SF407SGAF-BLU, Blu...",SF407SGAF-BLU,50051131277293,Anti Fog Lens:Yes; Brand:SecureFit; Case Quant...,https://www.3m.com/3M/en_US/company-us/all-3m-...
4,3M,Personal Protective Equipment,Dust and other Particles,"3M Particulate Respirator 8210, N95 160 EA/Case",8210,50051138464573,Acid Gas Reduction:No; Adjustable Buckle Strap...,https://www.3m.com/3M/en_US/company-us/all-3m-...


In [354]:
product_title = product_text.find('h1', {'class':'MMM--hdg'}).text
product_title

'3M™ SecureFit™ Safety Glasses SF401SGAF-RED, Red/Gray, Clear Scotchgard™\nAnti-fog Lens, 20 EA/Case'

In [327]:
df_total.product_line.unique()

array(['Safety Glasses', 'Dust and other Particles', 'Protective Eyewear',
       'PAPR Assembly', 'PAPR Systems', 'Healthcare Respirator',
       'Earplugs', 'Cartridge', 'Battery', 'PTT Adapter',
       'Battery & Charger', 'Half Facepiece', 'Filter', 'Fit Test Kit',
       'Belt', 'PAPR Kit', 'Head and Face Combinations', 'Safety Readers',
       'Battery Charger', 'Safety Goggle', 'Suspension for helmets',
       'Carrying Bag', 'Cable', 'Brow Pad', 'Battery Chargers',
       'Replacement Parts & Accessories'], dtype=object)

In [358]:
df_total.to_csv('output/3M_dftotal_2.csv', index=False)

# Text Cleaning

In [342]:
def remove_string(df, columns, string):
    for col in columns:
        df_total[col] = df_total[col].apply(lambda x: x.replace(string, ' '))

In [343]:
remove_string(df_total, ['product_title', 'description'], 'â„¢')

In [348]:
df_total.product_title.unique()

array(['3M SecureFit Safety Glasses SF401SGAF-RED, ...',
       '3M Particulate Respirator 8511, N95 80 EA/Case',
       '3M SecureFit Safety Glasses SF401SGAF-BLU, ...',
       '3M SecureFit Safety Glasses SF407SGAF-BLU, ...',
       '3M Particulate Respirator 8210, N95 160 EA/Case',
       '3M SecureFit Safety Glasses ...',
       '3M SecureFit Safety Glasses SF402SGAF-BLU, ...',
       '3M SecureFit Safety Glasses SF403SGAF-YEL, ...',
       '3M SecureFit Safety Glasses SF405SGAF-BLA, ...',
       '3M SecureFit Safety Glasses SF406SGAF-BLA, ...',
       '3M Versaflo  Powered Air Purifying Respirator PAPR ...',
       '3M Versaflo PAPR Assembly TR-614-HK, 1 ...',
       '3M Health Care Particulate Respirator and Surgical ...',
       '3M E-A-R Classic Earplugs 310-1001, ...',
       '3M Particulate Respirator 8210V, N95 80 EA/Case',
       '3M Aura Health Care Particulate Respirator and ...',
       '3M SecureFit Protective Eyewear SF401AF, Clear ...',
       '3M Versaflo Organic Vap

# Multiple Product Pages

In [332]:
len(multiple_product_list)

25

In [347]:
multiple_product_list

[('3Mâ„¢ Integrated Protective Eyewear',
  'https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Integrated-Protective-Eyewear/?N=5002385+8707795+3290055770&rt=rud'),
 ('3Mâ„¢ Virtuaâ„¢ Sport CCS Protective Eyewear',
  'https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Virtua-Sport-CCS-Protective-Eyewear/?N=5002385+8707795+8720539+3290055870&rt=rud'),
 ('3Mâ„¢ Virtuaâ„¢ VX6 Protective Eyewear',
  'https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-Virtua-VX6-Protective-Eyewear/?N=5002385+8707795+3288832842&rt=rud'),
 ('3Mâ„¢ SecureFitâ„¢ 200 Series Safety Glasses',
  'https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-SecureFit-200-Series-Safety-Glasses/?N=5002385+8707795+8720539+3290472100&rt=rud'),
 ('3Mâ„¢ W90 Series Faceshields & Visors',
  'https://www.3m.com/3M/en_US/company-us/all-3m-products/~/3M-W90-Series-Faceshields-Visors/?N=5002385+8707795+8720539+3288820895&rt=rud'),
 ('3Mâ„¢ Disposable Protective Coverall 4540',
  'https://www.3m.com/3M

# Debugging 

In [319]:
spec_table = product_text.find('div',{'class':'productSpecs'}).find_all('tr')
print(spec_table)
spec_table = spec_table[1:]
print(spec_table)
description = '; '.join([t.get_text().strip().replace('\n\n\n\n', ':') for t in spec_table])
print(description)

[<tr class="MMM--dat-row SNAPS--SpecTbl-RowOne SNAPS--hide">
<th scope="col">
<div class="MMM--dat-cell SNAPS-specAdjust SNAPS--SpecTbl-RowOneCols">
<span>Attribute Name</span>
</div>
</th><th scope="col">
<div class="dat-cell_specDefinition SNAPS-specAdjust SNAPS--SpecTbl-RowHdgCols">
<span>Value</span>
</div>
</th>
</tr>, <tr class="MMM--dat-row">
<td>
<div class="MMM--dat-cell SNAPS-specAdjust">
<span class="MMM--txt_bold">Acid Gas Reduction</span>
</div>
</td><td>
<div class="MMM--dat-cell dat-cell_specDefinition SNAPS-specAdjust SNAPS--SpecTbl-RowCols SNAPS--SpecTbl-Overflow">
<span class="MMM--txt_bold">No</span>
</div>
</td>
</tr>, <tr class="MMM--dat-row">
<td>
<div class="MMM--dat-cell SNAPS-specAdjust">
<span class="MMM--txt_bold">Adjustable Buckle Straps</span>
</div>
</td><td>
<div class="MMM--dat-cell dat-cell_specDefinition SNAPS-specAdjust SNAPS--SpecTbl-RowCols SNAPS--SpecTbl-Overflow">
<span class="MMM--txt_bold">No</span>
</div>
</td>
</tr>, <tr class="MMM--dat-row">


In [320]:
[t.get_text().strip().replace('\n\n\n\n', ':') for t in spec_table]

['Acid Gas Reduction:No',
 'Adjustable Buckle Straps:No',
 'Aerosol Type:Non-Oil',
 'Brand:3M™',
 'Cake-Resistant Filter Media:No',
 'Carbon Filter Material:No',
 'Case Quantity:160',
 'Class:Comfort',
 'Cool Flow™ Exhalation Valve:No',
 'Environment - Hot or Humid:No',
 'Exhalation Valve:No',
 'Faceseal:No',
 'FDA Cleared:No',
 'Features:Advanced Electrostatic Media',
 'Filter Class:N95',
 'Flame Resistance (ASTM D2859-96):No',
 'Fluid Resistant (ASTM F1862):N/A',
 'Folded:No',
 'Market:Aerospace, Defense, Homeland Security',
 'M-Noseclip:No',
 'N100 Required:No',
 'National Stock Number:4240014635449, 4240014292685',
 'Natural Rubber Latex Components:No',
 'Organic Vapor Reduction:No',
 'Packaging:Bulk Case',
 'Particle Types:Non-Oil',
 'Product Color:White',
 'Product Type:Dust and other Particles',
 'P-Series Filter:No',
 'Recommended Application:Airborne Biological Particles, Assembly and Mechanical, Bagging, Cleaning, Composite Finishing, Dry Chemical Handling, Final Finish, Foun

In [314]:
[a.get_text() for a in spec_table]

['\n\n\nAcid Gas Reduction\n\n\n\nNo\n\n\n',
 '\n\n\nAdjustable Buckle Straps\n\n\n\nNo\n\n\n',
 '\n\n\nAerosol Type\n\n\n\nNon-Oil\n\n\n',
 '\n\n\nBrand\n\n\n\n3M™\n\n\n',
 '\n\n\nCake-Resistant Filter Media\n\n\n\nNo\n\n\n',
 '\n\n\nCarbon Filter Material\n\n\n\nNo\n\n\n',
 '\n\n\nCase Quantity\n\n\n\n160\n\n\n',
 '\n\n\nClass\n\n\n\nComfort\n\n\n',
 '\n\n\nCool Flow™ Exhalation Valve\n\n\n\nNo\n\n\n',
 '\n\n\nEnvironment - Hot or Humid\n\n\n\nNo\n\n\n',
 '\n\n\nExhalation Valve\n\n\n\nNo\n\n\n',
 '\n\n\nFaceseal\n\n\n\nNo\n\n\n',
 '\n\n\nFDA Cleared\n\n\n\nNo\n\n\n',
 '\n\n\nFeatures\n\n\n\nAdvanced Electrostatic Media\n\n\n',
 '\n\n\nFilter Class\n\n\n\nN95\n\n\n',
 '\n\n\nFlame Resistance (ASTM D2859-96)\n\n\n\nNo\n\n\n',
 '\n\n\nFluid Resistant (ASTM F1862)\n\n\n\nN/A\n\n\n',
 '\n\n\nFolded\n\n\n\nNo\n\n\n',
 '\n\n\nMarket\n\n\n\nAerospace, Defense, Homeland Security\n\n\n',
 '\n\n\nM-Noseclip\n\n\n\nNo\n\n\n',
 '\n\n\nN100 Required\n\n\n\nNo\n\n\n',
 '\n\n\nNational Stock Number

In [51]:
x = product_text.find_all('span', {'class':'MMM--productSku'})
x

[<span class="MMM--productSku MMM--isNotOnMobile SNAPS--inlineBlock SNAPS--alternateId">3M Product Number SF401SGAF-RED</span>,
 <span class="MMM--productSku MMM--isNotOnMobile SNAPS--inlineBlock SNAPS--alternateId">3M ID 70071730512</span>,
 <span class="MMM--productSku MMM--isNotOnMobile SNAPS--inlineBlock SNAPS--alternateId">UPC 50051131277156</span>]

In [63]:
x = [x.get_text() for x in product_text.find_all('span', {'class':'MMM--productSku'})]
x

['3M Product Number\xa0SF401SGAF-RED',
 '3M ID\xa070071730512',
 'UPC\xa050051131277156']

In [89]:
y = [s.partition('\xa0') for s in x]
y

[('3M Product Number', '\xa0', 'SF401SGAF-RED'),
 ('3M ID', '\xa0', '70071730512'),
 ('UPC', '\xa0', '50051131277156')]

In [96]:
product_text.find('div',{'class':'MMM--contentDetailContain-bd'}).get_text().find('UPC')

153

In [203]:
product_text.find('div',{'class':'MMM--contentDetailContain-bd'})

<div class="MMM--contentDetailContain-bd">
<h1 class="MMM--hdg MMM--hdg_1 hdg_contentDetailTitle" itemprop="name" style="font-size:24px">3M™ SecureFit™ Safety Glasses SF401SGAF-RED, Red/Gray, Clear Scotchgard™
Anti-fog Lens, 20 EA/Case</h1>
<div>
<span class="MMM--productSku MMM--isNotOnMobile SNAPS--inlineBlock SNAPS--alternateId">3M Product Number SF401SGAF-RED</span>, <span class="MMM--productSku MMM--isNotOnMobile SNAPS--inlineBlock SNAPS--alternateId">3M ID 70071730512</span>, <span class="MMM--productSku MMM--isNotOnMobile SNAPS--inlineBlock SNAPS--alternateId">UPC 50051131277156</span>
</div>
<div class="MMM--lytJust"></div>
</div>

In [207]:
z = product_text.find('div',{'class':'MMM--contentDetailContain-bd'}).get_text().strip()
z = z.replace('\xa0', ': ').replace(',:', ' , ')
z

'3M™ SecureFit™ Safety Glasses SF401SGAF-RED, Red/Gray, Clear Scotchgard™\nAnti-fog Lens, 20 EA/Case\n\n3M Product Number: SF401SGAF-RED ,  3M ID: 70071730512 ,  UPC: 50051131277156'

In [210]:
a = re.search("(?<=Number: )(\S+)", z)
a.group()
# a
# re.search("(?<=name)name (\w+)", s)

'SF401SGAF-RED'

In [226]:
product_text.find('div',{'class':'productSpecs'})

<div class="productSpecs">
<table class="MMM--dat MMM--pdpTabVr SNAPS--SpecTbl-Mt15">
<tr class="MMM--dat-row SNAPS--SpecTbl-RowOne SNAPS--hide">
<th scope="col">
<div class="MMM--dat-cell SNAPS-specAdjust SNAPS--SpecTbl-RowOneCols">
<span>Attribute Name</span>
</div>
</th><th scope="col">
<div class="dat-cell_specDefinition SNAPS-specAdjust SNAPS--SpecTbl-RowHdgCols">
<span>Value</span>
</div>
</th>
</tr>
<tr class="MMM--dat-row">
<td>
<div class="MMM--dat-cell SNAPS-specAdjust">
<span class="MMM--txt_bold">Brand</span>
</div>
</td><td>
<div class="MMM--dat-cell dat-cell_specDefinition SNAPS-specAdjust SNAPS--SpecTbl-RowCols SNAPS--SpecTbl-Overflow">
<span class="MMM--txt_bold">SecureFit™</span>
</div>
</td>
</tr>
<tr class="MMM--dat-row">
<td>
<div class="MMM--dat-cell SNAPS-specAdjust">
<span class="MMM--txt_bold">Comfort Features</span>
</div>
</td><td>
<div class="MMM--dat-cell dat-cell_specDefinition SNAPS-specAdjust SNAPS--SpecTbl-RowCols SNAPS--SpecTbl-Overflow">
<span class="M

In [272]:
# description_list = []
spec_table = product_text.find('div',{'class':'productSpecs'}).find_all('tr')
spec_table = spec_table[1:]
# desc2 = ', '.join([t.get_text().strip().replace('\n', ':') for t in spec_table])
description = '; '.join([t.get_text().strip().replace('\n\n\n\n', ':') for t in spec_table])

# descrption = descrption.replace('Attribute Name:Value; ', '')
# for descr in description_table:
#     descr = descr.find_all('div', {'class':'attribute-group'})
#     desc1 = ', '.join([t.get_text().strip().replace('\n', '') for t in descr])
#     description_list.append(desc1 + ', ' + desc2)
# description_list
print(description)
re.search("(?<=Product Type:)(\S{0,10}\S{1,10})", description).group()

Brand:SecureFit™; Comfort Features:3M™ Pressure Diffusion Temple Technology; Eyewear Range:Value; Frame Material:Plastic; Frame Style:Frameless; Lens Coating:Anti-Scratch & Anti-Fog; Lens Color:Clear; Lens Marking:Z87+; Lens Material:Polycarbonate; Product Type:Safety Glasses; Recommended Industry:Automotive, Construction, General Manufacturing, Sub Applications, Mining, Oil & Gas, Pharmaceuticals, Primary Metals; SGBG New Product:Yes; Size:Unisex; Spare Part Type:Safety Glasses; Standards/Approvals:ANSI Z87.1-2015; Temple Color:Red/Gray; Temple Style:SecureFit™; UV Protection:99.9%


'Safety'

In [285]:
re.search("(?<=Product Type:)(.*)", description).group().split(';')[0]

'Safety Glasses'